# Advent of Code 2025
## Python Solutions

### Day 01: Secret Entrance

In [1]:
# part 1
from pathlib import Path

content = Path("d01.txt").read_text().splitlines()

ptr = 50
N = 100
counter = 0

def rotate(direction: str, number: int, ptr: int, counter: int) -> tuple[int, int]:
    if direction == "L":
        ptr =  (ptr - number) % N
    else: # direction == "R" 
        ptr =  (ptr + number) % N

    if ptr == 0:
        counter += 1

    return ptr, counter

for move in content:
    direction = move[:1]
    number = int(move[1:])

    ptr, counter = rotate(direction, number, ptr, counter)


counter

992

In [2]:
# part 2
content = Path("d01.txt").read_text().splitlines()
content

ptr = 50
N = 100
counter = 0

def rotate(direction: str, number: int, ptr: int, counter: int) -> tuple[int, int]:
    if direction ==  "L":
        for _ in range(number):
            ptr = (ptr - 1) % N
            if ptr == 0:
                counter += 1
    else: # direction == "R"
        for _ in range(number):
            ptr = (ptr + 1) % N
            if ptr == 0:
                counter += 1

    return ptr, counter

for move in content:
    direction = move[:1]
    number = int(move[1:])

    ptr, counter = rotate(direction, number, ptr, counter)

counter

6133

### Day 02: Gift Shop

In [3]:
# part 1
from pathlib import Path
content = Path("d02.txt").read_text()


# repeated twice:
# 55 (5 twice), 6464 (64 twice), 123123 (123 twice) are invalid IDs
def is_invalid_id(num: int) -> bool:
    s_num = str(num)
    if len(s_num) % 2 == 0:  # only even-length numbers can be repeated twice
        mid = len(s_num) // 2
        return s_num[:mid] == s_num[mid:]
    return False

def sum_invalid_ids(payload: str) -> int:
    ranges = payload.strip().split(",")
    total = 0
    for r in ranges:
        start, end = map(int, r.split("-"))
        for num in range(start, end + 1):
            if is_invalid_id(num):
                total += num
    return total

n_invalid = sum_invalid_ids(content)
n_invalid

30608905813

In [4]:
# part 2
content = Path("d02.txt").read_text()

# repeated at least twice: 
# 12341234 (1234 two times), 123123123 (123 three times), 1212121212 (12 five times), and 1111111 (1 seven times) are invalid IDs
def is_invalid_id(num: int) -> bool:
    s_num = str(num)
    length = len(s_num)
    # check all possible substring lengths
    for k in range(1, length // 2 + 1):
        if length % k == 0:
            if s_num == s_num[:k] * (length // k):
                return True
    return False

def sum_invalid_ids(payload: str) -> int:
    ranges = payload.strip().split(",")
    total = 0
    for r in ranges:
        start, end = map(int, r.split("-"))
        for num in range(start, end + 1):
            if is_invalid_id(num):
                total += num
    return total

n_invalid = sum_invalid_ids(content)
n_invalid

31898925685

**Alternative Solution: Regex**

Part 1: repeated twice
* Regex: `^(\d+)\1$`
* `(\d+)` captures one or more digits
* `\1` matches the same sequence again
* `^`...`$` anchors match the whole string

Part 2: at least twice
* Regex: `^(\d+)\1+$`
* same as above
* `\1+` captured group repeats one or more times

In [5]:
from typing import Callable
import re

pattern_part1 = re.compile(r'^(\d+)\1$')      # exactly two repeats
pattern_part2 = re.compile(r'^(\d+)\1+$')     # at least two repeats

def is_invalid1(num: int) -> bool:
    return bool(pattern_part1.match(str(num)))

def is_invalid2(num: int) -> bool:
    return bool(pattern_part2.match(str(num)))

def sum_invalid_ids(payload: str, valid_func: Callable[[int], bool]) -> int:
    ranges = payload.strip().split(',')
    total = 0
    for r in ranges:
        start, end = map(int, r.split('-'))
        for num in range(start, end + 1):
            if valid_func(num):
                total += num
    return total

content = Path("d02.txt").read_text()
n_invalid_1 = sum_invalid_ids(content, is_invalid1)
n_invalid_2 = sum_invalid_ids(content, is_invalid2)

print("Part 1: ", n_invalid_1)
print("Part 2: ", n_invalid_2)

Part 1:  30608905813
Part 2:  31898925685


### Day 03: Lobby

In [6]:
# part 1
from pathlib import Path

content = Path("d03.txt").read_text()


def sum_max_joltage(content: str) -> int:
    total = 0
    for line in content.splitlines():
        digits = list(map(int, line.strip()))
        # two largest digits in order
        max_jolt = 0
        for i in range(len(digits)):
            for j in range(i + 1, len(digits)):
                jolt = digits[i] * 10 + digits[j]
                if jolt > max_jolt:
                    max_jolt = jolt
        total += max_jolt
    return total

max_joltage = sum_max_joltage(content)
print(max_joltage)

17193


In [7]:
# part 2

content = Path("d03.txt").read_text()


def sum_max_joltage(content: str) -> int:
    total = 0
    k = 12  # number of digits to select
    for line in content.strip().splitlines():
        digits = list(map(int, line.strip()))
        drop = len(digits) - k
        stack = []
        for d in digits:
            while stack and drop > 0 and stack[-1] < d:
                stack.pop()
                drop -= 1
            stack.append(d)
        # Take only first k digits
        largest_digits = stack[:k]
        joltage = int(''.join(map(str, largest_digits)))
        total += joltage
    return total


max_joltage = sum_max_joltage(content)
print(max_joltage)

171297349921310


### Day 4: Printing Department

In [8]:
# part 1
from pathlib import Path
from itertools import product

def count_accessible_rolles(grid: list[str]) -> int:
    rows = range(len(grid))
    cols = range(len(grid[0]))
    positions = product(rows, cols)
    
    accessible_count = 0

    for (r, c) in positions:
        if grid[r][c] == "@":
            adjacent_rolls = 0

            
            for dr, dc in product([-1, 0, 1], repeat=2):
                if dr == 0 and dc == 0:
                    continue  # skip center cell
                nr, nc = r + dr, c + dc
                if nr in rows and nc in cols:
                    if grid[nr][nc] == "@":
                        adjacent_rolls += 1


            if adjacent_rolls < 4:
                accessible_count += 1
    
    return accessible_count

grid = Path("d04.txt").read_text().splitlines()

accessible_rolles = count_accessible_rolles(grid)
print(accessible_rolles)

1551


In [9]:
# part 2
def count_removed_rolls(grid: list[list[str]]) -> int:
    rows = range(len(grid))
    cols = range(len(grid[0]))
    positions = list(product(rows, cols))
    total_removed = 0

    while True:
        removed_this_round = 0

        for r, c in positions:
            if grid[r][c] == "@":
                adjacent_rolls = 0

                for dr, dc in product([-1, 0, 1], repeat=2):
                    if dr == 0 and dc == 0:
                        continue

                    nr, nc = r + dr, c + dc
                    if nr in rows and nc in cols:
                        if grid[nr][nc] == "@":
                            adjacent_rolls += 1

                if adjacent_rolls < 4:
                    grid[r][c] = "x"

        for r, c in positions:
            if grid[r][c] == "x":
                grid[r][c] = "."
                removed_this_round += 1

        total_removed += removed_this_round

        if removed_this_round == 0:
            break

    return total_removed


grid = Path("d04.txt").read_text().splitlines()
grid = [list(row) for row in grid]

removed_rolles = count_removed_rolls(grid)
print(removed_rolles)

9784
